<a href="https://colab.research.google.com/github/aadicodes/LLM-Chat/blob/main/Chatbot_Deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries
# !pip install transformers accelerate streamlit --quiet
!pip cache purge

!pip install --upgrade pip setuptools wheel


Files removed: 14 (3.2 MB)


In [2]:
!apt-get update
!apt-get install -y build-essential libssl-dev libffi-dev python3-dev


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,937 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,172 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jam

In [3]:
!pip install -q transformers accelerate text-generation

In [4]:
# check disk space
!df -h /
!du -sh /root/* /usr/* /content/* 2>/dev/null

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   40G   74G  35% /
199M	/usr/bin
25M	/usr/colab
20K	/usr/etc
4.0K	/usr/games
2.0M	/usr/grte
271M	/usr/include
5.5G	/usr/lib
15M	/usr/lib32
4.0K	/usr/lib64
437M	/usr/lib64-nvidia
1.7M	/usr/libexec
4.0K	/usr/libx32
26G	/usr/local
8.7M	/usr/sbin
502M	/usr/share
141M	/usr/src
55M	/content/sample_data


In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, pipeline
import torch

model_id = "deepseek-ai/deepseek-coder-6.7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
# Create chatbot pipeline
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [ ]:
# check disk space
!df -h /
!du -sh /root/* /usr/* /content/* 2>/dev/null

In [ ]:
# check the GPU allocated in virtual environment
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [15]:
def build_prompt(user_input, domain="healthcare"):
    system_prompt = f"You are a helpful assistant for {domain} customer service. Be accurate, empathetic, funny and concise."
    return f"{system_prompt}\nUser: {user_input}\nAssistant:"


In [16]:
def get_response(prompt, max_tokens=512):
    response = chatbot(prompt, max_new_tokens=max_tokens, do_sample=True, temperature=0.7)
    return response[0]['generated_text']


In [13]:
user_input = "I need help understanding my insurance coverage for a recent surgery."
prompt = build_prompt(user_input, domain="healthcare")
response = get_response(prompt)
print(response)


You are a helpful assistant for healthcare customer service. Be accurate, empathetic, and concise.
User: I need help understanding my insurance coverage for a recent surgery.
Assistant: I'm sorry for any confusion, but as an AI focused on healthcare, I'm unable to assist with insurance coverage questions. I'd recommend reaching out to your insurance company or a healthcare professional for assistance with these matters.



In [19]:
import gradio as gr

def build_prompt(user_input, domain="healthcare"):
    system_prompt = f"You are a helpful assistant for {domain} customer service. Be accurate, empathetic, funny and concise."
    return f"{system_prompt}\nUser: {user_input}\nAssistant:"

def get_response(prompt, max_tokens=512):
    # Assuming 'chatbot' pipeline is already defined and loaded
    # from the previous cell (3c370015)
    response = chatbot(prompt, max_new_tokens=max_tokens, do_sample=True, temperature=0.7)
    # Extract the generated text from the response.
    # The pipeline output includes the input prompt, so we need to remove it.
    generated_text = response[0]['generated_text']
    # Find the start of the assistant's response (after the prompt)
    assistant_start = generated_text.find("Assistant:")
    if assistant_start != -1:
        return generated_text[assistant_start + len("Assistant:"):].strip()
    return generated_text.strip()


def respond(user_input, chat_history, selected_domain):
    if not user_input:
        return chat_history, ""

    prompt = build_prompt(user_input, selected_domain)
    bot_response = get_response(prompt)

    # Append user input and bot response to chat history
    chat_history.append((user_input, bot_response))

    # Return the updated chat history and clear the input box
    return chat_history, ""


with gr.Blocks() as demo:
    gr.Markdown("## 💬 DeepSeek Chatbot for Finance & Healthcare - Rajesh Aadi")

    domain = gr.Radio(["finance", "healthcare"], label="Choose Domain", value="finance")

    # Use Chatbot component to display conversation history
    chatbot_output = gr.Chatbot(label="Conversation")

    # Use a State component to store chat history
    chat_history_state = gr.State([])

    with gr.Row():
        chat_input = gr.Textbox(label="Your question", scale=4)
        submit_btn = gr.Button("Send", scale=1)

    # Modify the click function to pass chat_history_state and update chatbot_output
    submit_btn.click(
        respond,
        inputs=[chat_input, chat_history_state, domain],
        outputs=[chatbot_output, chat_input]
    )

    # Add event listener to chat_input to trigger submit_btn.click on Enter key press
    chat_input.submit(
        respond,
        inputs=[chat_input, chat_history_state, domain],
        outputs=[chatbot_output, chat_input]
    )


demo.launch(share=True)

/tmp/ipython-input-3390332244.py:41: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_output = gr.Chatbot(label="Conversation")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9795c5212f03f778a7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Commented code below because Deepseek 33B loads 37B params out of 685B total,
# that demands serious memory capacity
#from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load DeepSeek V3.1 from Hugging Face
#model_id = "deepseek-ai/deepseek-coder-33b-instruct-GPTQ"
#tokenizer = AutoTokenizer.from_pretrained(model_id)
#model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")

# Create chatbot pipeline
#chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

""" Below code never worked , could not install GPTQ model on CUDA
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch

model_id = "deepseek-ai/deepseek-coder-33b-instruct-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
"""

In [ ]:
# Save this as app.py and run with: streamlit run app.py
"""
import streamlit as st

st.title("DeepSeek Chatbot")
domain = st.selectbox("Choose domain", ["healthcare", "finance"])
user_input = st.text_input("Ask your question")

if user_input:
    prompt = build_prompt(user_input, domain)
    response = get_response(prompt)
    st.write(response)
"""